In [29]:
import torch
from model.blur import BlurClass
from model.dnn import DnCNN
from model.jacob import jacobinNet
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm import tqdm
from dataset.dataset import get_matrix
from scipy.optimize import fminbound
from os import listdir,mkdir
from PIL.Image import open as imopen
import numpy as np
from os.path import join,isdir
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.io import savemat
now=datetime.now()

In [30]:
def iterAlgo(gt,xInit,y,numIter,dObj,rObj,gamma,tau):
    x=xInit
    s=xInit.clone()
    t = torch.tensor(1., dtype=torch.float32)
    for i in tqdm(range(numIter)):
        delta_g = dObj.grad(s, y)
        reg=tau * (rObj(s, False, False))
        xnext  =  torch.clamp(s - gamma * (delta_g.detach() + reg),0,1)
        tnext = 0.5*(1+torch.sqrt(1+4*t*t))
        s = xnext + ((t-1)/tnext)*(xnext-x)
        t = tnext.detach()
        x = xnext.detach()
    PSNR=psnr(x.detach().cpu().numpy(),gt.numpy(),data_range=1)
    return x,PSNR

In [31]:
modelPath='/export1/project/zihao/DEQ_Blur/outputs/pretrain-sigma_3_k-k4_sigma-2.55/2022-03-26-14-18-46/ckpts/epoch_0_34.52.pt'
kernelPath='/export1/project/zihao/DEQ_Blur/kernels/L09.mat'
kernelType='k2'
sigma=2.55
imagePath='/export1/project/zihao/DEQ_Blur/set3c'
patchSize=256
device='cpu'
gamma=1

In [32]:
modelDict=torch.load(modelPath,map_location='cpu')['model']
tau=modelDict['f.tau'].item()
del modelDict['f.tau']
for k in list(modelDict.keys()):
    modelDict[k[10:]]=modelDict[k]
    del modelDict[k]

In [33]:
k,kt=get_matrix(kernelPath,kernelType)


In [34]:
dObj=BlurClass(k,kt)
rObj=jacobinNet(DnCNN())
rObj.dnn.load_state_dict(modelDict)

<All keys matched successfully>

In [28]:
imNameList=listdir(imagePath)
if not isdir('outputs/optimizeTau'):
    mkdir('outputs/optimizeTau')
savePath=join('outputs/optimizeTau',now.strftime("%d-%b-%Y-%H-%M"))
mkdir(savePath)
for imName in imNameList:
    gt=torch.from_numpy(np.asarray(imopen(join(imagePath,imName)).transpose((2,0,1)))).float()/255.
    y=BlurClass.imfilter(gt, k)+torch.FloatTensor(gt.size()).normal_(0,std=sigma/255.)
    xInit=y.clone()
    f=lambda tauInit: -(iterAlgo(xInit,y,100,dObj,rObj,gamma,tauInit)[1])
    finalTau=fminbound(f, 0, tau, xtol = 5e-4, maxfun = 10, disp = 3)
    recon,PSNR=iterAlgo(gt,y,y,100,dObj,rObj,gamma,finalTau)
    saveName=imName.split('.')[0]
    plt.imsave(join(savePath,f'{saveName}_psnr{PSNR:.2f}.jpg'),recon.detach().cpu().numpy().transpose((1,2,0)))
    
    

IndentationError: expected an indented block (<ipython-input-28-534b352eff94>, line 3)